In [1]:
import numpy as np
import pandas as pd

from pandasql import sqldf, load_meat, load_births
pysqldf = lambda q: sqldf(q)

# Loading All DataSets

#ATTENTION, you need to unzip the listing.csv.zip!!

In [2]:
bcn_listings = pd.read_csv('listings.csv')

In [3]:
#A function is defined to convert the string variable for price with a dollar to a numerical variable.
def price_to_float(x):
    price = x.split("$")[1];
    if len(price.split(",")) == 2:
        price1 = int(price.split(",")[0])*1000
        price2 = float(price.split(",")[1])
        return price1 + price2
    else:
        return float(x.split("$")[1])

bcn_listings['price_mod'] = bcn_listings.apply(lambda x: price_to_float(x['price']),axis=1)

#The new dataset is defined
bcn_airbnb_dataset = bcn_listings[['id','property_type','host_id','host_name','host_response_time','price_mod','neighbourhood_group_cleansed',
                                   'review_scores_cleanliness','review_scores_location',
                                  ]]



bcn_airbnb_dataset.rename({'neighbourhood_group_cleansed': 'neighbourhood', 'price_mod': 'price',
                           'review_scores_cleanliness': 'review_cleanliness', 'review_scores_location': 'review_location',
                          
                          
                          }, axis=1, inplace=True)


/Users/josepferrersanchez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [4]:
#The table we are going to work with is finally defined. 
bcn_airbnb_dataset

,id,property_type,host_id,host_name,host_response_time,price,neighbourhood,review_cleanliness,review_location
0,18674,Entire rental unit,71615,Mireia And Maria,within an hour,220.0,Eixample,4.73,4.77
1,592553071373622034,Entire rental unit,115783949,Rosa María,within a few hours,61.0,Sants-Montjuïc,NaN,NaN
2,23289992,Private room in rental unit,173385037,Orlando,a few days or more,300.0,Sants-Montjuïc,NaN,NaN
3,32627616,Private room in rental unit,129000409,Olala Homes,within an hour,192.0,Sants-Montjuïc,5.00,4.50
4,50627552,Private room in hostel,370718107,Alexandre,within an hour,140.0,Sants-Montjuïc,4.93,4.36
...,...,...,...,...,...,...,...,...,...
16641,50050101,Private room in condo,143596103,Anika,within an hour,63.0,Sant Andreu,4.81,4.38
16642,50437306,Entire rental unit,158023606,Habitat Apartments,within an hour,208.0,Les Corts,NaN,NaN
16643,52610977,Private room in rental unit,425887052,Andrés,within an hour,25.0,Sants-Montjuïc,4.89,4.89
16644,20072386,Entire rental unit,45928842,Olga,within an hour,56.0,Sants-Montjuïc,4.72,4.62


# SQL Simple Queries

## Info by neighbourhood

In [5]:
pysqldf("""

        SELECT DISTINCT 
               neighbourhood,
               COUNT(DISTINCT id)                                                  AS num_apartments,
               COUNT(DISTINCT host_id)                                             AS num_hosts,
               ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
               ROUND(AVG(price),2)                                                 AS AVG_price,
               ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
               ROUND(AVG(review_location),2)                                       AS AVG_location    
        FROM bcn_airbnb_dataset 
        GROUP BY 1
        ORDER BY 2 DESC

        """
                    )

,neighbourhood,num_apartments,num_hosts,mean_apartments_by_host,AVG_price,AVG_cleanliness,AVG_location
0,Eixample,5561,2224,2.50,248.37,4.60,4.82
1,Ciutat Vella,3448,1735,1.99,153.56,4.53,4.79
2,Sants-Montjuïc,2105,1147,1.84,174.76,4.59,4.63
3,Sant Martí,1633,989,1.65,176.23,4.60,4.64
4,Gràcia,1436,753,1.91,164.56,4.62,4.78
5,Les Corts,723,373,1.94,203.72,4.53,4.46
6,Sarrià-Sant Gervasi,704,356,1.98,196.65,4.59,4.68
7,Horta-Guinardó,476,340,1.40,225.85,4.58,4.54
8,Sant Andreu,352,238,1.48,131.52,4.57,4.40
9,Nou Barris,208,156,1.33,80.22,4.68,4.51


## Info whole city

In [6]:
pysqldf("""

        SELECT DISTINCT 
               COUNT(DISTINCT id)                                                  AS num_apartments,
               COUNT(DISTINCT host_id)                                             AS num_hosts,
               ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
               ROUND(AVG(price),2)                                                 AS AVG_price,
               ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
               ROUND(AVG(review_location),2)                                       AS AVG_location    
        FROM bcn_airbnb_dataset 
        ORDER BY 1 DESC

        """
                    )

,num_apartments,num_hosts,mean_apartments_by_host,AVG_price,AVG_cleanliness,AVG_location
0,16646,7173,2.32,195.77,4.58,4.73


# SQL "Complex" Queries


## Non-Modular Version

In [7]:
pysqldf("""

        SELECT 
              T1.neighbourhood,
              T1.num_apartments,
              100*T1.num_apartments/T2.num_apartments AS comp_apartments,
              T1.num_hosts,
              100*T1.num_hosts/T2.num_hosts           AS comp_hosts,
              T1.mean_apartments_by_host,
              T1.AVG_price,
              CASE WHEN T1.AVG_price > T2.AVG_price  THEN "Above"
                   WHEN T1.AVG_price = T2.AVG_price  THEN "Equals"
                   WHEN T1.AVG_price < T2.AVG_price  THEN "Below"
              END AS "Price_comparison_to_city",
              T1.AVG_cleanliness,
              CASE WHEN T1.AVG_cleanliness > T2.AVG_cleanliness  THEN "Above"
                   WHEN T1.AVG_cleanliness = T2.AVG_cleanliness  THEN "Equals"
                   WHEN T1.AVG_cleanliness < T2.AVG_cleanliness  THEN "Below"
              END AS "Clean_Comparison_to_city",
              T1.AVG_location,
              CASE WHEN T1.AVG_location > T2.AVG_location  THEN "Above"
                   WHEN T1.AVG_location = T2.AVG_location  THEN "Equals"
                   WHEN T1.AVG_location < T2.AVG_location  THEN "Below"
              END AS "Location_Comparison_to_city"

              
        
        FROM 
            (
            SELECT DISTINCT 
                   neighbourhood,
                   COUNT(DISTINCT id)                                                  AS num_apartments,
                   COUNT(DISTINCT host_id)                                             AS num_hosts,
                   ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
                   ROUND(AVG(price),2)                                                 AS AVG_price,
                   ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
                   ROUND(AVG(review_location),2)                                       AS AVG_location    
            FROM bcn_airbnb_dataset 
            GROUP BY 1
            ORDER BY 2 DESC
            ) AS T1
            
         CROSS JOIN
            (
            SELECT DISTINCT 
                   COUNT(DISTINCT id)                                                  AS num_apartments,
                   COUNT(DISTINCT host_id)                                             AS num_hosts,
                   ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
                   ROUND(AVG(price),2)                                                 AS AVG_price,
                   ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
                   ROUND(AVG(review_location),2)                                       AS AVG_location    
            FROM bcn_airbnb_dataset 
            ORDER BY 1 DESC
            ) AS T2

        """
                    )

,neighbourhood,num_apartments,comp_apartments,num_hosts,comp_hosts,mean_apartments_by_host,AVG_price,Price_comparison_to_city,AVG_cleanliness,Clean_Comparison_to_city,AVG_location,Location_Comparison_to_city
0,Eixample,5561,33,2224,31,2.50,248.37,Above,4.60,Above,4.82,Above
1,Ciutat Vella,3448,20,1735,24,1.99,153.56,Below,4.53,Below,4.79,Above
2,Sants-Montjuïc,2105,12,1147,15,1.84,174.76,Below,4.59,Above,4.63,Below
3,Sant Martí,1633,9,989,13,1.65,176.23,Below,4.60,Above,4.64,Below
4,Gràcia,1436,8,753,10,1.91,164.56,Below,4.62,Above,4.78,Above
5,Les Corts,723,4,373,5,1.94,203.72,Above,4.53,Below,4.46,Below
6,Sarrià-Sant Gervasi,704,4,356,4,1.98,196.65,Above,4.59,Above,4.68,Below
7,Horta-Guinardó,476,2,340,4,1.40,225.85,Above,4.58,Equals,4.54,Below
8,Sant Andreu,352,2,238,3,1.48,131.52,Below,4.57,Below,4.40,Below
9,Nou Barris,208,1,156,2,1.33,80.22,Below,4.68,Above,4.51,Below


## Modular Version

In [8]:
pysqldf("""

        WITH 
        
        -- Structure------------------------------------------------------------------------------------------
        Neighbourhood_List AS 
        (
        
        SELECT DISTINCT 
               neighbourhood
               
        FROM bcn_airbnb_dataset
        
        ),
        
        
        -- Info BY neighbourhood ------------------------------------------------------------------------------------------
        Data_by_neighbourhood_BCN AS 
        (
        SELECT DISTINCT 
               neighbourhood,
               COUNT(DISTINCT id)                                                  AS num_apartments,
               COUNT(DISTINCT host_id)                                             AS num_hosts,
               ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
               ROUND(AVG(price),2)                                                 AS AVG_price,
               ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
               ROUND(AVG(review_location),2)                                       AS AVG_location    
        FROM bcn_airbnb_dataset 
        GROUP BY 1
        ORDER BY 2 DESC
        ),
        
        -- Global Info------------------------------------------------------------------------------------------
        
        Data_global_BCN AS 
        (
        
        SELECT DISTINCT 
               COUNT(DISTINCT id)                                                  AS num_apartments,
               COUNT(DISTINCT host_id)                                             AS num_hosts,
               ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
               ROUND(AVG(price),2)                                                 AS AVG_price,
               ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
               ROUND(AVG(review_location),2)                                       AS AVG_location    
        FROM bcn_airbnb_dataset 
        ORDER BY 1 DESC
        )
        
        -- Final Query----------------------------------------------------------------------------------------

        SELECT 
              Structure.neighbourhood,
              T1.num_apartments,
              100*T1.num_apartments/T2.num_apartments AS comp_apartments,
              T1.num_hosts,
              100*T1.num_hosts/T2.num_hosts           AS comp_hosts,
              T1.mean_apartments_by_host,
              T1.AVG_price,
              CASE WHEN T1.AVG_price > T2.AVG_price  THEN "Above"
                   WHEN T1.AVG_price = T2.AVG_price  THEN "Equals"
                   WHEN T1.AVG_price < T2.AVG_price  THEN "Below"
              END AS "Price_comparison_to_city",
              T1.AVG_cleanliness,
              CASE WHEN T1.AVG_cleanliness > T2.AVG_cleanliness  THEN "Above"
                   WHEN T1.AVG_cleanliness = T2.AVG_cleanliness  THEN "Equals"
                   WHEN T1.AVG_cleanliness < T2.AVG_cleanliness  THEN "Below"
              END AS "Clean_Comparison_to_city",
              T1.AVG_location,
              CASE WHEN T1.AVG_location > T2.AVG_location  THEN "Above"
                   WHEN T1.AVG_location = T2.AVG_location  THEN "Equals"
                   WHEN T1.AVG_location < T2.AVG_location  THEN "Below"
              END AS "Location_Comparison_to_city"

              
        
        FROM Neighbourhood_List              AS Structure
        LEFT JOIN Data_by_neighbourhood_BCN  AS T1
          ON Structure.neighbourhood = T1.neighbourhood 
        CROSS JOIN Data_global_BCN           AS T2

        """
                    )

,neighbourhood,num_apartments,comp_apartments,num_hosts,comp_hosts,mean_apartments_by_host,AVG_price,Price_comparison_to_city,AVG_cleanliness,Clean_Comparison_to_city,AVG_location,Location_Comparison_to_city
0,Eixample,5561,33,2224,31,2.50,248.37,Above,4.60,Above,4.82,Above
1,Sants-Montjuïc,2105,12,1147,15,1.84,174.76,Below,4.59,Above,4.63,Below
2,Gràcia,1436,8,753,10,1.91,164.56,Below,4.62,Above,4.78,Above
3,Ciutat Vella,3448,20,1735,24,1.99,153.56,Below,4.53,Below,4.79,Above
4,Sant Martí,1633,9,989,13,1.65,176.23,Below,4.60,Above,4.64,Below
5,Sarrià-Sant Gervasi,704,4,356,4,1.98,196.65,Above,4.59,Above,4.68,Below
6,Les Corts,723,4,373,5,1.94,203.72,Above,4.53,Below,4.46,Below
7,Horta-Guinardó,476,2,340,4,1.40,225.85,Above,4.58,Equals,4.54,Below
8,Sant Andreu,352,2,238,3,1.48,131.52,Below,4.57,Below,4.40,Below
9,Nou Barris,208,1,156,2,1.33,80.22,Below,4.68,Above,4.51,Below


# Added Features

In [9]:
pysqldf("""

        WITH 
        
        --****************************************************************************************************
        --INFO BY neighbourhood
        --****************************************************************************************************
        -- Structure------------------------------------------------------------------------------------------
        Neighbourhood_List AS 
        (
        
        SELECT DISTINCT 
               neighbourhood
               
        FROM bcn_airbnb_dataset
        
        ),

        --MOST COMMON APARTMENT TYPE BY neighbourhood---------------------------------------------------------
        Apartment_Property_Type_By_Neighbourhood AS 
        (
        SELECT DISTINCT 
               neighbourhood,
               property_type,
               COUNT(DISTINCT id)                                                  AS num_apartments
               
        FROM bcn_airbnb_dataset 
        GROUP BY 1,2
        ORDER BY 3 DESC
        ),
        
        Most_Common_Apartment_Property_Type_By_Neighbourhood AS 
        
        (
        SELECT neighbourhood,
               property_type,
               MAX(num_apartments) AS max_num
        
        FROM Apartment_Property_Type_By_Neighbourhood
        GROUP BY 1
        ORDER BY 3 DESC
        ),
        
        --All Info about Hosts in Barcelona-------------------------------------------------------------------
        Hosts_BCN AS 
        (
        SELECT DISTINCT 
               neighbourhood,
               COUNT(DISTINCT host_id)                                             AS num_hosts,
               COUNT(DISTINCT id)                                                  AS num_apartments,
               ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host
               
        FROM bcn_airbnb_dataset 
        GROUP BY 1
        ORDER BY 2 DESC
        ),
        
        --Average price and reviews in BCN by neighbourhood---------------------------------------------------

        AVG_price_reviews_BCN_by_neighbourhood AS 
        (
        SELECT DISTINCT 
               neighbourhood,
               ROUND(AVG(price),2)                                                 AS AVG_price,
               ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
               ROUND(AVG(review_location),2)                                       AS AVG_location    
        FROM bcn_airbnb_dataset 
        GROUP BY 1
        ORDER BY 2 DESC
        ),
        
        --****************************************************************************************************
        --GLOBAL INFO BCN
        --****************************************************************************************************
        
        Data_global_BCN AS 
        (
        
        SELECT DISTINCT 
               COUNT(DISTINCT id)                                                  AS num_apartments,
               COUNT(DISTINCT host_id)                                             AS num_hosts,
               ROUND(CAST(COUNT(DISTINCT id) AS FLOAT)/COUNT(DISTINCT host_id),2)  AS mean_apartments_by_host,
               ROUND(AVG(price),2)                                                 AS AVG_price,
               ROUND(AVG(review_cleanliness),2)                                    AS AVG_cleanliness,
               ROUND(AVG(review_location),2)                                       AS AVG_location    
        FROM bcn_airbnb_dataset 
        ORDER BY 1 DESC
        )
        
        --****************************************************************************************************
        --FINAL QUERY
        --****************************************************************************************************
        
        SELECT 
        -- Main Structure
        Structure.neighbourhood,
        -- Hosts Info
        Hosts.num_hosts,
        100*ROUND(CAST(Hosts.num_hosts AS FLOAT)/Global.num_hosts,2)               AS ptg_hosts,           
        Hosts.num_apartments,
        100*ROUND(CAST(Hosts.num_apartments AS FLOAT)/Global.num_apartments,2)     AS ptg_apartments,  
        Hosts.mean_apartments_by_host                                              AS ap_by_host,
        -- Common Apartment
        Common_Apartment.property_type                                             AS most_common,
        100*ROUND(CAST(Common_Apartment.max_num AS FLOAT)/Hosts.num_apartments,2)  AS ptg_most_com,
        -- AVG price and reviews
        AVG_by_neighbourhood.AVG_price,
        AVG_by_neighbourhood.AVG_cleanliness,
        AVG_by_neighbourhood.AVG_location
        
        
        FROM Neighbourhood_List                                            AS Structure
        LEFT JOIN Hosts_BCN                                                AS Hosts
          ON Structure.neighbourhood = Hosts.neighbourhood
        LEFT JOIN Most_Common_Apartment_Property_Type_By_Neighbourhood     AS Common_Apartment
          ON Structure.neighbourhood = Common_Apartment.neighbourhood
        LEFT JOIN AVG_price_reviews_BCN_by_neighbourhood                   AS AVG_by_neighbourhood
          ON Structure.neighbourhood = AVG_by_neighbourhood.neighbourhood
        CROSS JOIN Data_global_BCN                                         AS Global
        ORDER BY Hosts.num_hosts DESC
        """
)

,neighbourhood,num_hosts,ptg_hosts,num_apartments,ptg_apartments,ap_by_host,most_common,ptg_most_com,AVG_price,AVG_cleanliness,AVG_location
0,Eixample,2224,31.0,5561,33.0,2.50,Entire rental unit,55.0,248.37,4.60,4.82
1,Ciutat Vella,1735,24.0,3448,21.0,1.99,Entire rental unit,45.0,153.56,4.53,4.79
2,Sants-Montjuïc,1147,16.0,2105,13.0,1.84,Entire rental unit,50.0,174.76,4.59,4.63
3,Sant Martí,989,14.0,1633,10.0,1.65,Entire rental unit,46.0,176.23,4.60,4.64
4,Gràcia,753,10.0,1436,9.0,1.91,Entire rental unit,50.0,164.56,4.62,4.78
5,Les Corts,373,5.0,723,4.0,1.94,Entire rental unit,46.0,203.72,4.53,4.46
6,Sarrià-Sant Gervasi,356,5.0,704,4.0,1.98,Entire rental unit,44.0,196.65,4.59,4.68
7,Horta-Guinardó,340,5.0,476,3.0,1.40,Private room in rental unit,46.0,225.85,4.58,4.54
8,Sant Andreu,238,3.0,352,2.0,1.48,Private room in rental unit,45.0,131.52,4.57,4.40
9,Nou Barris,156,2.0,208,1.0,1.33,Private room in rental unit,65.0,80.22,4.68,4.51
